In [85]:
import pygmt
import numpy as np
import pandas as pd
import geopy.distance

In [86]:
def pgd(
        mag_w,
        distance_km,
        *,
        depth=0,
        method='Melgar2D_2015'):
    """
    pgd Calculates expected peak ground displacment based on hypo-central
    distance and Mw. PGD is defined as sqrt(x^2+y^2+z^2) for all methods
    except Melgar2D_2015, where it is sqrt(x^2+y^2).
    Distance units are in km.
    pgd is returned in cm.

    Parameters
    ----------
    :param mag_w: float
        Magnitude of earthquake
    :param distance_km: float or list of floats
        distance from earthquake, assumed to be epicentral distance unless a
        depth is supplied.
    :param depth: float
        optional
        depth of earthquake in km, depth positive, default is 0 km
    :param method: string
        optional
        method used for pgd calculation, can be
        Melgar2D_2015 (default)
        Melgar3D_2015
        Crowell_2016
        Ruhl_2019 (Do not use...)
    :return:
        float or list of floats (if distance is supplied as an list), output is in cm.

    UNAVCO 2020-2022 (Mencin)
    """
    distance = np.array(distance_km)

    if method == 'Melgar2D_2015':
        a = -4.639
        b = 1.063
        c = -0.137
    elif method == 'Melgar3D_2015':
        a = -4.434
        b = 1.047
        c = -0.138
    elif method == 'Crowell_2016':
        a = -6.687
        b = 1.500
        c = -0.214
    elif method == 'Ruhl_2019':
        a = -5.919
        b = 1.009
        c = -0.145
        distance_km = distance_km*1000
    else:
        sys.exit("Invalid method, use: Melgar2D_2015 (default), Melgar3D_2015, Crowell_2016 or Ruhl_2019.")

    log_pgd = a + b * mag_w + c * mag_w * np.log10(np.sqrt(np.power(distance, 2) + np.power(depth, 2)))
    peak_ground_displacement_mm = np.float_power(10, log_pgd)

    if method == 'Ruhl_2019':
        peak_ground_displacement_mm = peak_ground_displacement_mm * 100
    else:
        peak_ground_displacement_mm = peak_ground_displacement_mm

    return peak_ground_displacement_mm

In [87]:
colombia_stations = 'Colombia-Noise.csv'
ecuador_stations = 'Ecuador-Noise.csv'
earthquakes_file = 'Earthquakes.csv'
cities_file = 'Cities.csv'

In [88]:
# Read in station locations and noise for each station
dfE = pd.read_csv(ecuador_stations)
dfC = pd.read_csv(colombia_stations)
frames = [dfC, dfE ]
df = pd.concat(frames)

In [89]:
cities = pd.read_csv(cities_file)

In [90]:
eqs = pd.read_csv(earthquakes_file)
eqs['Date'] = pd.to_datetime(eqs['Date'])

In [91]:
#minimum number of stations for detection
min = 4
#Seimic velocity in km/sec
vel = 4
# Time required to process and issue warning
issue = 6

In [92]:
cities

,Latitude,Longitude,City
0,4.7110,-74.0721,Bogota
1,6.2476,-75.5658,Medellin
2,3.4516,-76.5320,Cali
3,11.0041,-74.8070,Barranquilla
4,10.3932,-75.4832,Cartagena


In [93]:
# Make a table that shows the expected PGD in each city from each earthquake
pgds = eqs.copy()

for i in cities.itertuples():
    pgds[i[3]+'_PGD cm'] = eqs.apply(lambda row: pgd(row.Mw,
                                     geopy.distance.geodesic( [i[1], i[2]],(row.Latitude,row.Longitude)).km),
                                     axis=1)
pgds       

,Latitude,Longitude,Depth,Date,Mw,Bogota_PGD cm,Medellin_PGD cm,Cali_PGD cm,Barranquilla_PGD cm,Cartagena_PGD cm
0,4.800,-77.180,19.1,1991-11-19,7.2,3.246280,4.641924,6.696227,1.540635,1.747456
1,4.720,-77.570,16.0,2004-11-15,7.2,2.889964,4.001353,6.114357,1.491579,1.690577
2,2.849,-74.798,55.0,2067-02-09,7.0,3.576820,2.099699,3.865502,0.928982,0.997092
3,0.955,-79.369,25.0,2006-01-31,8.8,18.628978,18.591910,35.786679,9.869482,10.842945
4,0.025,-79.955,25.0,2042-05-14,7.8,3.392828,3.354778,5.432202,2.043594,2.205222
5,1.000,-79.400,60.0,2058-01-19,7.8,3.969284,3.973861,7.091090,2.266372,2.464401
6,0.380,-79.950,20.0,2016-04-16,7.7,3.004805,3.000654,4.898114,1.816812,1.963593
7,2.320,-78.810,19.7,1979-12-12,8.1,7.893155,8.280664,17.841305,4.124305,4.561166


In [94]:
pgds.to_csv('PGDTable.csv', float_format="%.1f")

In [95]:
def blind_spot(
        eq_lat,
        eq_lon,
        city_lat,
        city_lon,
        df,
        mini):
    df['distance']   = df.apply(lambda row: geopy.distance.geodesic((eq_lat,eq_lon),(row.lat,row.lon)).km, axis=1)
    df.sort_values(by='distance', inplace=True, ignore_index=True)
    dect_dist = df.loc[mini-1,'distance']
    return dect_dist

In [96]:
def warning_time(
        eq_lat,
        eq_lon,
        city_lat,
        city_lon,
        df,
        mini,
        velocity,
        detect_t):
    df['distance']   = df.apply(lambda row: geopy.distance.geodesic((eq_lat,eq_lon),(row.lat,row.lon)).km, axis=1)
    df.sort_values(by='distance', inplace=True, ignore_index=True)
    dect_dist = df.loc[mini-1,'distance']
    city_dist = geopy.distance.geodesic((eq_lat,eq_lon),(city_lat,city_lon)).km
    warning_time_sec = ((city_dist - dect_dist)/velocity) - detect_t
    return warning_time_sec

In [97]:
# make a table for the blind spot diameter (the blind spot is the diameter of first detcion at a specifc minium)
# these should be city independent
blind = eqs.copy()
for i in cities.itertuples():
    blind[i[3]+'_BlindSpot km'] = eqs.apply(lambda row: blind_spot(row.Latitude, row.Longitude, i[1], i[2], df, min), axis=1)
blind

,Latitude,Longitude,Depth,Date,Mw,Bogota_BlindSpot km,Medellin_BlindSpot km,Cali_BlindSpot km,Barranquilla_BlindSpot km,Cartagena_BlindSpot km
0,4.800,-77.180,19.1,1991-11-19,7.2,127.368726,127.368726,127.368726,127.368726,127.368726
1,4.720,-77.570,16.0,2004-11-15,7.2,152.377377,152.377377,152.377377,152.377377,152.377377
2,2.849,-74.798,55.0,2067-02-09,7.0,75.907627,75.907627,75.907627,75.907627,75.907627
3,0.955,-79.369,25.0,2006-01-31,8.8,39.612788,39.612788,39.612788,39.612788,39.612788
4,0.025,-79.955,25.0,2042-05-14,7.8,52.225923,52.225923,52.225923,52.225923,52.225923
5,1.000,-79.400,60.0,2058-01-19,7.8,36.814605,36.814605,36.814605,36.814605,36.814605
6,0.380,-79.950,20.0,2016-04-16,7.7,45.291075,45.291075,45.291075,45.291075,45.291075
7,2.320,-78.810,19.7,1979-12-12,8.1,105.618102,105.618102,105.618102,105.618102,105.618102


In [98]:
blind.to_csv('BlindSpotTable.csv', float_format="%.1f")

In [99]:
# make a table for the Warning times
warn = eqs.copy()
for i in cities.itertuples():
    warn[i[3]+'_Warning sec'] = eqs.apply(lambda row: warning_time(row.Latitude, row.Longitude, i[1], i[2], df, min, vel, issue), axis=1)
warn

,Latitude,Longitude,Depth,Date,Mw,Bogota_Warning sec,Medellin_Warning sec,Cali_Warning sec,Barranquilla_Warning sec,Cartagena_Warning sec
0,4.800,-77.180,19.1,1991-11-19,7.2,48.389544,22.166448,3.547065,145.733910,123.725834
1,4.720,-77.570,16.0,2004-11-15,7.2,52.924680,25.663665,1.290537,145.603963,122.985762
2,2.849,-74.798,55.0,2067-02-09,7.0,30.303715,71.362375,26.005799,200.495936,184.459614
3,0.955,-79.369,25.0,2006-01-31,8.8,164.238120,164.535985,88.914322,289.207627,266.306339
4,0.025,-79.955,25.0,2042-05-14,7.8,189.569403,191.782945,115.243727,316.218818,293.167386
5,1.000,-79.400,60.0,2058-01-19,7.8,164.929178,164.735026,89.453639,289.129806,266.183990
6,0.380,-79.950,20.0,2016-04-16,7.7,185.231514,185.497179,110.136980,309.021589,285.850307
7,2.320,-78.810,19.7,1979-12-12,8.1,114.861487,108.636557,38.217280,231.918511,208.992789


In [100]:
warn.to_csv('WarningTable.csv', float_format="%.1f")

In [101]:
df25 = pd.read_csv('Sim-25km-Noise.csv')
frames = [dfC, dfE, df25]
df = pd.concat(frames)

In [102]:
# make a table for the blind spot diameter (the blind spot is the diameter of first detcion at a specifc minium)
# these should be city independent
blind25 = eqs.copy()
for i in cities.itertuples():
    blind25[i[3]+'_BlindSpot_(25km) km'] = eqs.apply(lambda row: blind_spot(row.Latitude, row.Longitude, i[1], i[2], df, min), axis=1)
blind25

,Latitude,Longitude,Depth,Date,Mw,Bogota_BlindSpot_(25km) km,Medellin_BlindSpot_(25km) km,Cali_BlindSpot_(25km) km,Barranquilla_BlindSpot_(25km) km,Cartagena_BlindSpot_(25km) km
0,4.800,-77.180,19.1,1991-11-19,7.2,36.858061,36.858061,36.858061,36.858061,36.858061
1,4.720,-77.570,16.0,2004-11-15,7.2,37.364977,37.364977,37.364977,37.364977,37.364977
2,2.849,-74.798,55.0,2067-02-09,7.0,75.907627,75.907627,75.907627,75.907627,75.907627
3,0.955,-79.369,25.0,2006-01-31,8.8,39.612788,39.612788,39.612788,39.612788,39.612788
4,0.025,-79.955,25.0,2042-05-14,7.8,52.225923,52.225923,52.225923,52.225923,52.225923
5,1.000,-79.400,60.0,2058-01-19,7.8,36.814605,36.814605,36.814605,36.814605,36.814605
6,0.380,-79.950,20.0,2016-04-16,7.7,45.291075,45.291075,45.291075,45.291075,45.291075
7,2.320,-78.810,19.7,1979-12-12,8.1,36.335531,36.335531,36.335531,36.335531,36.335531


In [103]:
blind25.to_csv('BlindSpot25Table.csv', float_format="%.1f")

In [104]:
# make a table for the Warning times
warn25 = eqs.copy()
for i in cities.itertuples():
    warn25[i[3]+'_Warning_(25km) sec'] = eqs.apply(lambda row: warning_time(row.Latitude, row.Longitude, i[1], i[2], df, min, vel, issue), axis=1)
warn25

,Latitude,Longitude,Depth,Date,Mw,Bogota_Warning_(25km) sec,Medellin_Warning_(25km) sec,Cali_Warning_(25km) sec,Barranquilla_Warning_(25km) sec,Cartagena_Warning_(25km) sec
0,4.800,-77.180,19.1,1991-11-19,7.2,71.017211,44.794114,26.174732,168.361576,146.353501
1,4.720,-77.570,16.0,2004-11-15,7.2,81.677780,54.416765,30.043637,174.357063,151.738862
2,2.849,-74.798,55.0,2067-02-09,7.0,30.303715,71.362375,26.005799,200.495936,184.459614
3,0.955,-79.369,25.0,2006-01-31,8.8,164.238120,164.535985,88.914322,289.207627,266.306339
4,0.025,-79.955,25.0,2042-05-14,7.8,189.569403,191.782945,115.243727,316.218818,293.167386
5,1.000,-79.400,60.0,2058-01-19,7.8,164.929178,164.735026,89.453639,289.129806,266.183990
6,0.380,-79.950,20.0,2016-04-16,7.7,185.231514,185.497179,110.136980,309.021589,285.850307
7,2.320,-78.810,19.7,1979-12-12,8.1,132.182130,125.957200,55.537923,249.239154,226.313432


In [105]:
warn25.to_csv('Warning25Table.csv', float_format="%.1f")

In [106]:
df50 = pd.read_csv('Sim-50km-Noise.csv')
frames = [dfC, dfE, df50]
df = pd.concat(frames)

In [107]:
# make a table for the blind spot diameter (the blind spot is the diameter of first detcion at a specifc minium)
# these should be city independent
blind50 = eqs.copy()
for i in cities.itertuples():
    blind50[i[3]+'_BlindSpot_(50km) km'] = eqs.apply(lambda row: blind_spot(row.Latitude, row.Longitude, i[1], i[2], df, min), axis=1)
blind50

,Latitude,Longitude,Depth,Date,Mw,Bogota_BlindSpot_(50km) km,Medellin_BlindSpot_(50km) km,Cali_BlindSpot_(50km) km,Barranquilla_BlindSpot_(50km) km,Cartagena_BlindSpot_(50km) km
0,4.800,-77.180,19.1,1991-11-19,7.2,62.851950,62.851950,62.851950,62.851950,62.851950
1,4.720,-77.570,16.0,2004-11-15,7.2,57.704034,57.704034,57.704034,57.704034,57.704034
2,2.849,-74.798,55.0,2067-02-09,7.0,75.907627,75.907627,75.907627,75.907627,75.907627
3,0.955,-79.369,25.0,2006-01-31,8.8,39.612788,39.612788,39.612788,39.612788,39.612788
4,0.025,-79.955,25.0,2042-05-14,7.8,52.225923,52.225923,52.225923,52.225923,52.225923
5,1.000,-79.400,60.0,2058-01-19,7.8,36.814605,36.814605,36.814605,36.814605,36.814605
6,0.380,-79.950,20.0,2016-04-16,7.7,45.291075,45.291075,45.291075,45.291075,45.291075
7,2.320,-78.810,19.7,1979-12-12,8.1,56.023500,56.023500,56.023500,56.023500,56.023500


In [108]:
blind50.to_csv('BlindSpot50Table.csv', float_format="%.1f")

In [109]:
# make a table for the Warning times
warn50 = eqs.copy()
for i in cities.itertuples():
    warn50[i[3]+'_Warning_(50km) sec'] = eqs.apply(lambda row: warning_time(row.Latitude, row.Longitude, i[1], i[2], df, min, vel, issue), axis=1)
warn50

,Latitude,Longitude,Depth,Date,Mw,Bogota_Warning_(50km) sec,Medellin_Warning_(50km) sec,Cali_Warning_(50km) sec,Barranquilla_Warning_(50km) sec,Cartagena_Warning_(50km) sec
0,4.800,-77.180,19.1,1991-11-19,7.2,64.518738,38.295642,19.676260,161.863104,139.855028
1,4.720,-77.570,16.0,2004-11-15,7.2,76.593016,49.332001,24.958872,169.272298,146.654098
2,2.849,-74.798,55.0,2067-02-09,7.0,30.303715,71.362375,26.005799,200.495936,184.459614
3,0.955,-79.369,25.0,2006-01-31,8.8,164.238120,164.535985,88.914322,289.207627,266.306339
4,0.025,-79.955,25.0,2042-05-14,7.8,189.569403,191.782945,115.243727,316.218818,293.167386
5,1.000,-79.400,60.0,2058-01-19,7.8,164.929178,164.735026,89.453639,289.129806,266.183990
6,0.380,-79.950,20.0,2016-04-16,7.7,185.231514,185.497179,110.136980,309.021589,285.850307
7,2.320,-78.810,19.7,1979-12-12,8.1,127.260137,121.035208,50.615930,244.317161,221.391440


In [110]:
warn50.to_csv('Warning50Table.csv', float_format="%.1f")

In [111]:
df100 = pd.read_csv('Sim-100km-Noise.csv')
frames = [dfC, dfE, df100]
df = pd.concat(frames)

In [112]:
# make a table for the blind spot diameter (the blind spot is the diameter of first detcion at a specifc minium)
# these should be city independent
blind100 = eqs.copy()
for i in cities.itertuples():
    blind100[i[3]+'_BlindSpot_(100km) km'] = eqs.apply(lambda row: blind_spot(row.Latitude, row.Longitude, i[1], i[2], df, min), axis=1)
blind100

,Latitude,Longitude,Depth,Date,Mw,Bogota_BlindSpot_(100km) km,Medellin_BlindSpot_(100km) km,Cali_BlindSpot_(100km) km,Barranquilla_BlindSpot_(100km) km,Cartagena_BlindSpot_(100km) km
0,4.800,-77.180,19.1,1991-11-19,7.2,85.427290,85.427290,85.427290,85.427290,85.427290
1,4.720,-77.570,16.0,2004-11-15,7.2,86.453890,86.453890,86.453890,86.453890,86.453890
2,2.849,-74.798,55.0,2067-02-09,7.0,75.907627,75.907627,75.907627,75.907627,75.907627
3,0.955,-79.369,25.0,2006-01-31,8.8,39.612788,39.612788,39.612788,39.612788,39.612788
4,0.025,-79.955,25.0,2042-05-14,7.8,52.225923,52.225923,52.225923,52.225923,52.225923
5,1.000,-79.400,60.0,2058-01-19,7.8,36.814605,36.814605,36.814605,36.814605,36.814605
6,0.380,-79.950,20.0,2016-04-16,7.7,45.291075,45.291075,45.291075,45.291075,45.291075
7,2.320,-78.810,19.7,1979-12-12,8.1,56.062831,56.062831,56.062831,56.062831,56.062831


In [113]:
blind100.to_csv('BlindSpot100Table.csv', float_format="%.1f")

In [114]:
# make a table for the Warning times
warn100 = eqs.copy()
for i in cities.itertuples():
    warn100[i[3]+'_Warning_(100km) sec'] = eqs.apply(lambda row: warning_time(row.Latitude, row.Longitude, i[1], i[2], df, min, vel, issue), axis=1)
warn100

,Latitude,Longitude,Depth,Date,Mw,Bogota_Warning_(100km) sec,Medellin_Warning_(100km) sec,Cali_Warning_(100km) sec,Barranquilla_Warning_(100km) sec,Cartagena_Warning_(100km) sec
0,4.800,-77.180,19.1,1991-11-19,7.2,58.874903,32.651807,14.032425,156.219269,134.211193
1,4.720,-77.570,16.0,2004-11-15,7.2,69.405552,42.144537,17.771408,162.084834,139.466634
2,2.849,-74.798,55.0,2067-02-09,7.0,30.303715,71.362375,26.005799,200.495936,184.459614
3,0.955,-79.369,25.0,2006-01-31,8.8,164.238120,164.535985,88.914322,289.207627,266.306339
4,0.025,-79.955,25.0,2042-05-14,7.8,189.569403,191.782945,115.243727,316.218818,293.167386
5,1.000,-79.400,60.0,2058-01-19,7.8,164.929178,164.735026,89.453639,289.129806,266.183990
6,0.380,-79.950,20.0,2016-04-16,7.7,185.231514,185.497179,110.136980,309.021589,285.850307
7,2.320,-78.810,19.7,1979-12-12,8.1,127.250305,121.025375,50.606097,244.307329,221.381607


In [115]:
warn100.to_csv('Warning100Table.csv', float_format="%.1f")